In [1]:
"""from google.colab import drive
drive.mount('/content/drive')"""

"from google.colab import drive\ndrive.mount('/content/drive')"

In [2]:
import pandas as pd
df_train = pd.read_csv('/content/drive/MyDrive/SMM4H 2024/labeled_training (2).csv')
df_dev = pd.read_csv('/content/drive/MyDrive/SMM4H 2024/SMM4H_2024_Task6_Dev.csv')
df_test = pd.read_csv('/content/drive/MyDrive/SMM4H 2024/unlabeled_testing.csv')

In [3]:
print(len(df_train))
print(len(df_dev))
print(len(df_test))

8800
4200
8200


In [4]:
import re
import pandas as pd
def preprocess_text(text):
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    text = text.lower()
    text = re.sub(r'[@]\w+', '', text)
    text = re.sub(r'[^\x00-\x7F]+', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text
df_train['text'] = df_train['text'].apply(preprocess_text)
df_dev['text'] = df_dev['text'].apply(preprocess_text)
df_test['text'] = df_test['text'].apply(preprocess_text)

In [5]:
!pip install transformers seqeval torch tqdm accelerate sentencepiece datasets

In [6]:
print(df_train['source'].value_counts())
print(df_dev['source'].value_counts())
print(df_test['source'].value_counts())

source
twitter    8800
Name: count, dtype: int64
source
twitter    2200
reddit     2000
Name: count, dtype: int64
source
reddit     6000
twitter    2200
Name: count, dtype: int64


In [7]:
df_train_reddit = df_dev[df_dev['source'] == 'reddit']
df_train_twitter = pd.concat([df_train, df_dev[df_dev['source'] == 'twitter']])
df_test_reddit = df_test[df_test['source'] == 'reddit']
df_test_twitter = df_test[df_test['source'] == 'twitter']

In [8]:
print(len(df_train_reddit))
print(len(df_train_twitter))
print(len(df_test_reddit))
print(len(df_test_twitter))

2000
11000
6000
2200


In [9]:
from datasets import load_metric
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, IntervalStrategy, TrainerCallback
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset
from accelerate import Accelerator
import numpy as np
import torch
import os
class TextDataset(Dataset):
    def __init__(self, tokenizer, texts, labels):
        self.encodings = tokenizer(texts, truncation=True, padding=True, max_length=512)
        self.labels = labels
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item
    def __len__(self):
        return len(self.labels)
class SaveModelCallback(TrainerCallback):
    def __init__(self, output_dir):
        self.output_dir = output_dir
    def on_epoch_end(self, args, state, control, **kwargs):
        output_dir = os.path.join(self.output_dir, f"epoch_bartlarge_A_{state.epoch}")
        os.makedirs(output_dir, exist_ok=True)
        model.save_pretrained(output_dir)
        tokenizer.save_pretrained(output_dir)
texts = df_train_reddit['text'].tolist()
labels = df_train_reddit['label'].tolist()
tokenizer = AutoTokenizer.from_pretrained('facebook/bart-large')
model = AutoModelForSequenceClassification.from_pretrained('facebook/bart-large', num_labels=2) # Adjust num_labels as per your requirement
train_dataset = TextDataset(tokenizer, texts, labels)
training_args = TrainingArguments(
    output_dir='./results_temp',
    num_train_epochs=20,
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=0,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=100,
    evaluation_strategy=IntervalStrategy.NO,
    save_strategy=IntervalStrategy.NO,
    load_best_model_at_end=False,
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    callbacks=[SaveModelCallback(output_dir='./model_saves')]
)
trainer.train()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of BartForSequenceClassification were not initialized from the model checkpoint at facebook/bart-large and are newly initialized: ['classification_head.dense.bias', 'classification_head.dense.weight', 'classification_head.out_proj.bias', 'classification_head.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWar

Step,Training Loss
100,0.477400
200,0.347800
300,0.249500
400,0.165000
500,0.155500
600,0.116100
700,0.061200
800,0.115000
900,0.067200
1000,0.070700


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file

TrainOutput(global_step=5000, training_loss=0.04850020657004789, metrics={'train_runtime': 4548.9061, 'train_samples_per_second': 8.793, 'train_steps_per_second': 1.099, 'total_flos': 4.347131879424e+16, 'train_loss': 0.04850020657004789, 'epoch': 20.0})

In [10]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.utils.data import Dataset, DataLoader
from tqdm.auto import tqdm
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, recall_score, precision_score
import matplotlib.pyplot as plt
import seaborn as sns
class TextDataset(Dataset):
    def __init__(self, tokenizer, texts, labels=None):
        self.encodings = tokenizer(texts, truncation=True, padding=True, max_length=512)
        self.labels = labels if labels is not None else [0] * len(texts)
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels is not None:
            item['labels'] = torch.tensor(self.labels[idx])
        return item
    def __len__(self):
        return len(self.encodings['input_ids'])

In [12]:
tokenizer = AutoTokenizer.from_pretrained('facebook/bart-large')
model = AutoModelForSequenceClassification.from_pretrained('/content/model_saves/epoch_bartlarge_A_1.0')                       #  A/B 1.0/20.0
model = model.to('cuda')
dev_texts = df_test_reddit['text'].tolist()                                                                                    # reddit/twitter
dev_dataset = TextDataset(tokenizer, dev_texts)
dev_loader = DataLoader(dev_dataset, batch_size=4, shuffle=False)
predictions = []
for batch in tqdm(dev_loader, desc="Evaluating"):
    inputs = {k: v.to(model.device) for k, v in batch.items() if k != 'labels'}
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    probabilities = torch.nn.functional.softmax(logits, dim=1)
    pred_labels = logits.argmax(1)
    predictions.extend(zip(pred_labels.cpu().numpy(), probabilities[:,0].cpu().numpy(), probabilities[:,1].cpu().numpy()))
df_predictions_1 = pd.DataFrame(predictions, columns=['predicted_label', 'prediction_0_weight', 'prediction_1_weight'])       #_1.0/_20.0
df_test_reddit.reset_index(drop=True, inplace=True)                                                                           # reddit/twitter
df_predictions_1.reset_index(drop=True, inplace=True)                                                                         #_1.0/_20.0
df_merged_1 = pd.concat([df_test_reddit, df_predictions_1], axis=1)                                                           # reddit/twitter #_1.0/_20.0 #_1.0/_20.0
df_merged_1.to_csv('BartLargeA_task6_predictions_069_epoch_1.csv', index=False)                                               # A/B            #_1.0/_20.0 #_1/_20

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'LABEL_0', '1': 'LABEL_1'}. The number of labels wil be overwritten to 2.


Evaluating:   0%|          | 0/1500 [00:00<?, ?it/s]

In [13]:
tokenizer = AutoTokenizer.from_pretrained('facebook/bart-large')
model = AutoModelForSequenceClassification.from_pretrained('/content/model_saves/epoch_bartlarge_A_2.0')                       #  A/B 1.0/20.0
model = model.to('cuda')
dev_texts = df_test_reddit['text'].tolist()                                                                                    # reddit/twitter
dev_dataset = TextDataset(tokenizer, dev_texts)
dev_loader = DataLoader(dev_dataset, batch_size=4, shuffle=False)
predictions = []
for batch in tqdm(dev_loader, desc="Evaluating"):
    inputs = {k: v.to(model.device) for k, v in batch.items() if k != 'labels'}
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    probabilities = torch.nn.functional.softmax(logits, dim=1)
    pred_labels = logits.argmax(1)
    predictions.extend(zip(pred_labels.cpu().numpy(), probabilities[:,0].cpu().numpy(), probabilities[:,1].cpu().numpy()))
df_predictions_2 = pd.DataFrame(predictions, columns=['predicted_label', 'prediction_0_weight', 'prediction_1_weight'])       #_1.0/_20.0
df_test_reddit.reset_index(drop=True, inplace=True)                                                                           # reddit/twitter
df_predictions_2.reset_index(drop=True, inplace=True)                                                                         #_1.0/_20.0
df_merged_2 = pd.concat([df_test_reddit, df_predictions_2], axis=1)                                                           # reddit/twitter #_1.0/_20.0 #_1.0/_20.0
df_merged_2.to_csv('BartLargeA_task6_predictions_069_epoch_2.csv', index=False)                                               # A/B            #_1.0/_20.0 #_1/_20

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'LABEL_0', '1': 'LABEL_1'}. The number of labels wil be overwritten to 2.


Evaluating:   0%|          | 0/1500 [00:00<?, ?it/s]

In [14]:
tokenizer = AutoTokenizer.from_pretrained('facebook/bart-large')
model = AutoModelForSequenceClassification.from_pretrained('/content/model_saves/epoch_bartlarge_A_3.0')                       #  A/B 1.0/20.0
model = model.to('cuda')
dev_texts = df_test_reddit['text'].tolist()                                                                                    # reddit/twitter
dev_dataset = TextDataset(tokenizer, dev_texts)
dev_loader = DataLoader(dev_dataset, batch_size=4, shuffle=False)
predictions = []
for batch in tqdm(dev_loader, desc="Evaluating"):
    inputs = {k: v.to(model.device) for k, v in batch.items() if k != 'labels'}
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    probabilities = torch.nn.functional.softmax(logits, dim=1)
    pred_labels = logits.argmax(1)
    predictions.extend(zip(pred_labels.cpu().numpy(), probabilities[:,0].cpu().numpy(), probabilities[:,1].cpu().numpy()))
df_predictions_3 = pd.DataFrame(predictions, columns=['predicted_label', 'prediction_0_weight', 'prediction_1_weight'])       #_1.0/_20.0
df_test_reddit.reset_index(drop=True, inplace=True)                                                                           # reddit/twitter
df_predictions_3.reset_index(drop=True, inplace=True)                                                                         #_1.0/_20.0
df_merged_3 = pd.concat([df_test_reddit, df_predictions_3], axis=1)                                                           # reddit/twitter #_1.0/_20.0 #_1.0/_20.0
df_merged_3.to_csv('BartLargeA_task6_predictions_069_epoch_3.csv', index=False)                                               # A/B            #_1.0/_20.0 #_1/_20

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'LABEL_0', '1': 'LABEL_1'}. The number of labels wil be overwritten to 2.


Evaluating:   0%|          | 0/1500 [00:00<?, ?it/s]

In [15]:
tokenizer = AutoTokenizer.from_pretrained('facebook/bart-large')
model = AutoModelForSequenceClassification.from_pretrained('/content/model_saves/epoch_bartlarge_A_4.0')                       #  A/B 1.0/20.0
model = model.to('cuda')
dev_texts = df_test_reddit['text'].tolist()                                                                                    # reddit/twitter
dev_dataset = TextDataset(tokenizer, dev_texts)
dev_loader = DataLoader(dev_dataset, batch_size=4, shuffle=False)
predictions = []
for batch in tqdm(dev_loader, desc="Evaluating"):
    inputs = {k: v.to(model.device) for k, v in batch.items() if k != 'labels'}
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    probabilities = torch.nn.functional.softmax(logits, dim=1)
    pred_labels = logits.argmax(1)
    predictions.extend(zip(pred_labels.cpu().numpy(), probabilities[:,0].cpu().numpy(), probabilities[:,1].cpu().numpy()))
df_predictions_4 = pd.DataFrame(predictions, columns=['predicted_label', 'prediction_0_weight', 'prediction_1_weight'])       #_1.0/_20.0
df_test_reddit.reset_index(drop=True, inplace=True)                                                                           # reddit/twitter
df_predictions_4.reset_index(drop=True, inplace=True)                                                                         #_1.0/_20.0
df_merged_4 = pd.concat([df_test_reddit, df_predictions_4], axis=1)                                                           # reddit/twitter #_1.0/_20.0 #_1.0/_20.0
df_merged_4.to_csv('BartLargeA_task6_predictions_069_epoch_4.csv', index=False)                                               # A/B            #_1.0/_20.0 #_1/_20

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'LABEL_0', '1': 'LABEL_1'}. The number of labels wil be overwritten to 2.


Evaluating:   0%|          | 0/1500 [00:00<?, ?it/s]

In [16]:
tokenizer = AutoTokenizer.from_pretrained('facebook/bart-large')
model = AutoModelForSequenceClassification.from_pretrained('/content/model_saves/epoch_bartlarge_A_5.0')                       #  A/B 1.0/20.0
model = model.to('cuda')
dev_texts = df_test_reddit['text'].tolist()                                                                                    # reddit/twitter
dev_dataset = TextDataset(tokenizer, dev_texts)
dev_loader = DataLoader(dev_dataset, batch_size=4, shuffle=False)
predictions = []
for batch in tqdm(dev_loader, desc="Evaluating"):
    inputs = {k: v.to(model.device) for k, v in batch.items() if k != 'labels'}
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    probabilities = torch.nn.functional.softmax(logits, dim=1)
    pred_labels = logits.argmax(1)
    predictions.extend(zip(pred_labels.cpu().numpy(), probabilities[:,0].cpu().numpy(), probabilities[:,1].cpu().numpy()))
df_predictions_5 = pd.DataFrame(predictions, columns=['predicted_label', 'prediction_0_weight', 'prediction_1_weight'])       #_1.0/_20.0
df_test_reddit.reset_index(drop=True, inplace=True)                                                                           # reddit/twitter
df_predictions_5.reset_index(drop=True, inplace=True)                                                                         #_1.0/_20.0
df_merged_5 = pd.concat([df_test_reddit, df_predictions_5], axis=1)                                                           # reddit/twitter #_1.0/_20.0 #_1.0/_20.0
df_merged_5.to_csv('BartLargeA_task6_predictions_069_epoch_5.csv', index=False)                                               # A/B            #_1.0/_20.0 #_1/_20

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'LABEL_0', '1': 'LABEL_1'}. The number of labels wil be overwritten to 2.


Evaluating:   0%|          | 0/1500 [00:00<?, ?it/s]

In [17]:
tokenizer = AutoTokenizer.from_pretrained('facebook/bart-large')
model = AutoModelForSequenceClassification.from_pretrained('/content/model_saves/epoch_bartlarge_A_6.0')                       #  A/B 1.0/20.0
model = model.to('cuda')
dev_texts = df_test_reddit['text'].tolist()                                                                                    # reddit/twitter
dev_dataset = TextDataset(tokenizer, dev_texts)
dev_loader = DataLoader(dev_dataset, batch_size=4, shuffle=False)
predictions = []
for batch in tqdm(dev_loader, desc="Evaluating"):
    inputs = {k: v.to(model.device) for k, v in batch.items() if k != 'labels'}
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    probabilities = torch.nn.functional.softmax(logits, dim=1)
    pred_labels = logits.argmax(1)
    predictions.extend(zip(pred_labels.cpu().numpy(), probabilities[:,0].cpu().numpy(), probabilities[:,1].cpu().numpy()))
df_predictions_6 = pd.DataFrame(predictions, columns=['predicted_label', 'prediction_0_weight', 'prediction_1_weight'])       #_1.0/_20.0
df_test_reddit.reset_index(drop=True, inplace=True)                                                                           # reddit/twitter
df_predictions_6.reset_index(drop=True, inplace=True)                                                                         #_1.0/_20.0
df_merged_6 = pd.concat([df_test_reddit, df_predictions_6], axis=1)                                                           # reddit/twitter #_1.0/_20.0 #_1.0/_20.0
df_merged_6.to_csv('BartLargeA_task6_predictions_069_epoch_6.csv', index=False)                                               # A/B            #_1.0/_20.0 #_1/_20

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'LABEL_0', '1': 'LABEL_1'}. The number of labels wil be overwritten to 2.


Evaluating:   0%|          | 0/1500 [00:00<?, ?it/s]

In [18]:
tokenizer = AutoTokenizer.from_pretrained('facebook/bart-large')
model = AutoModelForSequenceClassification.from_pretrained('/content/model_saves/epoch_bartlarge_A_7.0')                       #  A/B 1.0/20.0
model = model.to('cuda')
dev_texts = df_test_reddit['text'].tolist()                                                                                    # reddit/twitter
dev_dataset = TextDataset(tokenizer, dev_texts)
dev_loader = DataLoader(dev_dataset, batch_size=4, shuffle=False)
predictions = []
for batch in tqdm(dev_loader, desc="Evaluating"):
    inputs = {k: v.to(model.device) for k, v in batch.items() if k != 'labels'}
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    probabilities = torch.nn.functional.softmax(logits, dim=1)
    pred_labels = logits.argmax(1)
    predictions.extend(zip(pred_labels.cpu().numpy(), probabilities[:,0].cpu().numpy(), probabilities[:,1].cpu().numpy()))
df_predictions_7 = pd.DataFrame(predictions, columns=['predicted_label', 'prediction_0_weight', 'prediction_1_weight'])       #_1.0/_20.0
df_test_reddit.reset_index(drop=True, inplace=True)                                                                           # reddit/twitter
df_predictions_7.reset_index(drop=True, inplace=True)                                                                         #_1.0/_20.0
df_merged_7 = pd.concat([df_test_reddit, df_predictions_7], axis=1)                                                           # reddit/twitter #_1.0/_20.0 #_1.0/_20.0
df_merged_7.to_csv('BartLargeA_task6_predictions_069_epoch_7.csv', index=False)                                               # A/B            #_1.0/_20.0 #_1/_20

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'LABEL_0', '1': 'LABEL_1'}. The number of labels wil be overwritten to 2.


Evaluating:   0%|          | 0/1500 [00:00<?, ?it/s]

In [19]:
tokenizer = AutoTokenizer.from_pretrained('facebook/bart-large')
model = AutoModelForSequenceClassification.from_pretrained('/content/model_saves/epoch_bartlarge_A_8.0')                       #  A/B 1.0/20.0
model = model.to('cuda')
dev_texts = df_test_reddit['text'].tolist()                                                                                    # reddit/twitter
dev_dataset = TextDataset(tokenizer, dev_texts)
dev_loader = DataLoader(dev_dataset, batch_size=4, shuffle=False)
predictions = []
for batch in tqdm(dev_loader, desc="Evaluating"):
    inputs = {k: v.to(model.device) for k, v in batch.items() if k != 'labels'}
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    probabilities = torch.nn.functional.softmax(logits, dim=1)
    pred_labels = logits.argmax(1)
    predictions.extend(zip(pred_labels.cpu().numpy(), probabilities[:,0].cpu().numpy(), probabilities[:,1].cpu().numpy()))
df_predictions_8 = pd.DataFrame(predictions, columns=['predicted_label', 'prediction_0_weight', 'prediction_1_weight'])       #_1.0/_20.0
df_test_reddit.reset_index(drop=True, inplace=True)                                                                           # reddit/twitter
df_predictions_8.reset_index(drop=True, inplace=True)                                                                         #_1.0/_20.0
df_merged_8 = pd.concat([df_test_reddit, df_predictions_8], axis=1)                                                           # reddit/twitter #_1.0/_20.0 #_1.0/_20.0
df_merged_8.to_csv('BartLargeA_task6_predictions_069_epoch_8.csv', index=False)                                               # A/B            #_1.0/_20.0 #_1/_20

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'LABEL_0', '1': 'LABEL_1'}. The number of labels wil be overwritten to 2.


Evaluating:   0%|          | 0/1500 [00:00<?, ?it/s]

In [20]:
tokenizer = AutoTokenizer.from_pretrained('facebook/bart-large')
model = AutoModelForSequenceClassification.from_pretrained('/content/model_saves/epoch_bartlarge_A_9.0')                       #  A/B 1.0/20.0
model = model.to('cuda')
dev_texts = df_test_reddit['text'].tolist()                                                                                    # reddit/twitter
dev_dataset = TextDataset(tokenizer, dev_texts)
dev_loader = DataLoader(dev_dataset, batch_size=4, shuffle=False)
predictions = []
for batch in tqdm(dev_loader, desc="Evaluating"):
    inputs = {k: v.to(model.device) for k, v in batch.items() if k != 'labels'}
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    probabilities = torch.nn.functional.softmax(logits, dim=1)
    pred_labels = logits.argmax(1)
    predictions.extend(zip(pred_labels.cpu().numpy(), probabilities[:,0].cpu().numpy(), probabilities[:,1].cpu().numpy()))
df_predictions_9 = pd.DataFrame(predictions, columns=['predicted_label', 'prediction_0_weight', 'prediction_1_weight'])       #_1.0/_20.0
df_test_reddit.reset_index(drop=True, inplace=True)                                                                           # reddit/twitter
df_predictions_9.reset_index(drop=True, inplace=True)                                                                         #_1.0/_20.0
df_merged_9 = pd.concat([df_test_reddit, df_predictions_9], axis=1)                                                           # reddit/twitter #_1.0/_20.0 #_1.0/_20.0
df_merged_9.to_csv('BartLargeA_task6_predictions_069_epoch_9.csv', index=False)                                               # A/B            #_1.0/_20.0 #_1/_20

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'LABEL_0', '1': 'LABEL_1'}. The number of labels wil be overwritten to 2.


Evaluating:   0%|          | 0/1500 [00:00<?, ?it/s]

In [21]:
tokenizer = AutoTokenizer.from_pretrained('facebook/bart-large')
model = AutoModelForSequenceClassification.from_pretrained('/content/model_saves/epoch_bartlarge_A_10.0')                       #  A/B 1.0/20.0
model = model.to('cuda')
dev_texts = df_test_reddit['text'].tolist()                                                                                    # reddit/twitter
dev_dataset = TextDataset(tokenizer, dev_texts)
dev_loader = DataLoader(dev_dataset, batch_size=4, shuffle=False)
predictions = []
for batch in tqdm(dev_loader, desc="Evaluating"):
    inputs = {k: v.to(model.device) for k, v in batch.items() if k != 'labels'}
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    probabilities = torch.nn.functional.softmax(logits, dim=1)
    pred_labels = logits.argmax(1)
    predictions.extend(zip(pred_labels.cpu().numpy(), probabilities[:,0].cpu().numpy(), probabilities[:,1].cpu().numpy()))
df_predictions_10 = pd.DataFrame(predictions, columns=['predicted_label', 'prediction_0_weight', 'prediction_1_weight'])       #_1.0/_20.0
df_test_reddit.reset_index(drop=True, inplace=True)                                                                           # reddit/twitter
df_predictions_10.reset_index(drop=True, inplace=True)                                                                         #_1.0/_20.0
df_merged_10 = pd.concat([df_test_reddit, df_predictions_10], axis=1)                                                           # reddit/twitter #_1.0/_20.0 #_1.0/_20.0
df_merged_10.to_csv('BartLargeA_task6_predictions_069_epoch_10.csv', index=False)                                               # A/B            #_1.0/_20.0 #_1/_20

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'LABEL_0', '1': 'LABEL_1'}. The number of labels wil be overwritten to 2.


Evaluating:   0%|          | 0/1500 [00:00<?, ?it/s]

In [22]:
tokenizer = AutoTokenizer.from_pretrained('facebook/bart-large')
model = AutoModelForSequenceClassification.from_pretrained('/content/model_saves/epoch_bartlarge_A_11.0')                       #  A/B 1.0/20.0
model = model.to('cuda')
dev_texts = df_test_reddit['text'].tolist()                                                                                    # reddit/twitter
dev_dataset = TextDataset(tokenizer, dev_texts)
dev_loader = DataLoader(dev_dataset, batch_size=4, shuffle=False)
predictions = []
for batch in tqdm(dev_loader, desc="Evaluating"):
    inputs = {k: v.to(model.device) for k, v in batch.items() if k != 'labels'}
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    probabilities = torch.nn.functional.softmax(logits, dim=1)
    pred_labels = logits.argmax(1)
    predictions.extend(zip(pred_labels.cpu().numpy(), probabilities[:,0].cpu().numpy(), probabilities[:,1].cpu().numpy()))
df_predictions_11 = pd.DataFrame(predictions, columns=['predicted_label', 'prediction_0_weight', 'prediction_1_weight'])       #_1.0/_20.0
df_test_reddit.reset_index(drop=True, inplace=True)                                                                           # reddit/twitter
df_predictions_11.reset_index(drop=True, inplace=True)                                                                         #_1.0/_20.0
df_merged_11 = pd.concat([df_test_reddit, df_predictions_11], axis=1)                                                           # reddit/twitter #_1.0/_20.0 #_1.0/_20.0
df_merged_11.to_csv('BartLargeA_task6_predictions_069_epoch_11.csv', index=False)                                               # A/B            #_1.0/_20.0 #_1/_20

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'LABEL_0', '1': 'LABEL_1'}. The number of labels wil be overwritten to 2.


Evaluating:   0%|          | 0/1500 [00:00<?, ?it/s]

In [23]:
tokenizer = AutoTokenizer.from_pretrained('facebook/bart-large')
model = AutoModelForSequenceClassification.from_pretrained('/content/model_saves/epoch_bartlarge_A_12.0')                       #  A/B 1.0/20.0
model = model.to('cuda')
dev_texts = df_test_reddit['text'].tolist()                                                                                    # reddit/twitter
dev_dataset = TextDataset(tokenizer, dev_texts)
dev_loader = DataLoader(dev_dataset, batch_size=4, shuffle=False)
predictions = []
for batch in tqdm(dev_loader, desc="Evaluating"):
    inputs = {k: v.to(model.device) for k, v in batch.items() if k != 'labels'}
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    probabilities = torch.nn.functional.softmax(logits, dim=1)
    pred_labels = logits.argmax(1)
    predictions.extend(zip(pred_labels.cpu().numpy(), probabilities[:,0].cpu().numpy(), probabilities[:,1].cpu().numpy()))
df_predictions_12 = pd.DataFrame(predictions, columns=['predicted_label', 'prediction_0_weight', 'prediction_1_weight'])       #_1.0/_20.0
df_test_reddit.reset_index(drop=True, inplace=True)                                                                           # reddit/twitter
df_predictions_12.reset_index(drop=True, inplace=True)                                                                         #_1.0/_20.0
df_merged_12 = pd.concat([df_test_reddit, df_predictions_12], axis=1)                                                           # reddit/twitter #_1.0/_20.0 #_1.0/_20.0
df_merged_12.to_csv('BartLargeA_task6_predictions_069_epoch_12.csv', index=False)                                               # A/B            #_1.0/_20.0 #_1/_20

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'LABEL_0', '1': 'LABEL_1'}. The number of labels wil be overwritten to 2.


Evaluating:   0%|          | 0/1500 [00:00<?, ?it/s]

In [24]:
tokenizer = AutoTokenizer.from_pretrained('facebook/bart-large')
model = AutoModelForSequenceClassification.from_pretrained('/content/model_saves/epoch_bartlarge_A_13.0')                       #  A/B 1.0/20.0
model = model.to('cuda')
dev_texts = df_test_reddit['text'].tolist()                                                                                    # reddit/twitter
dev_dataset = TextDataset(tokenizer, dev_texts)
dev_loader = DataLoader(dev_dataset, batch_size=4, shuffle=False)
predictions = []
for batch in tqdm(dev_loader, desc="Evaluating"):
    inputs = {k: v.to(model.device) for k, v in batch.items() if k != 'labels'}
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    probabilities = torch.nn.functional.softmax(logits, dim=1)
    pred_labels = logits.argmax(1)
    predictions.extend(zip(pred_labels.cpu().numpy(), probabilities[:,0].cpu().numpy(), probabilities[:,1].cpu().numpy()))
df_predictions_13 = pd.DataFrame(predictions, columns=['predicted_label', 'prediction_0_weight', 'prediction_1_weight'])       #_1.0/_20.0
df_test_reddit.reset_index(drop=True, inplace=True)                                                                           # reddit/twitter
df_predictions_13.reset_index(drop=True, inplace=True)                                                                         #_1.0/_20.0
df_merged_13 = pd.concat([df_test_reddit, df_predictions_13], axis=1)                                                           # reddit/twitter #_1.0/_20.0 #_1.0/_20.0
df_merged_13.to_csv('BartLargeA_task6_predictions_069_epoch_13.csv', index=False)                                               # A/B            #_1.0/_20.0 #_1/_20

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'LABEL_0', '1': 'LABEL_1'}. The number of labels wil be overwritten to 2.


Evaluating:   0%|          | 0/1500 [00:00<?, ?it/s]

In [25]:
tokenizer = AutoTokenizer.from_pretrained('facebook/bart-large')
model = AutoModelForSequenceClassification.from_pretrained('/content/model_saves/epoch_bartlarge_A_14.0')                       #  A/B 1.0/20.0
model = model.to('cuda')
dev_texts = df_test_reddit['text'].tolist()                                                                                    # reddit/twitter
dev_dataset = TextDataset(tokenizer, dev_texts)
dev_loader = DataLoader(dev_dataset, batch_size=4, shuffle=False)
predictions = []
for batch in tqdm(dev_loader, desc="Evaluating"):
    inputs = {k: v.to(model.device) for k, v in batch.items() if k != 'labels'}
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    probabilities = torch.nn.functional.softmax(logits, dim=1)
    pred_labels = logits.argmax(1)
    predictions.extend(zip(pred_labels.cpu().numpy(), probabilities[:,0].cpu().numpy(), probabilities[:,1].cpu().numpy()))
df_predictions_14 = pd.DataFrame(predictions, columns=['predicted_label', 'prediction_0_weight', 'prediction_1_weight'])       #_1.0/_20.0
df_test_reddit.reset_index(drop=True, inplace=True)                                                                           # reddit/twitter
df_predictions_14.reset_index(drop=True, inplace=True)                                                                         #_1.0/_20.0
df_merged_14 = pd.concat([df_test_reddit, df_predictions_14], axis=1)                                                           # reddit/twitter #_1.0/_20.0 #_1.0/_20.0
df_merged_14.to_csv('BartLargeA_task6_predictions_069_epoch_14.csv', index=False)                                               # A/B            #_1.0/_20.0 #_1/_20

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'LABEL_0', '1': 'LABEL_1'}. The number of labels wil be overwritten to 2.


Evaluating:   0%|          | 0/1500 [00:00<?, ?it/s]

In [26]:
tokenizer = AutoTokenizer.from_pretrained('facebook/bart-large')
model = AutoModelForSequenceClassification.from_pretrained('/content/model_saves/epoch_bartlarge_A_15.0')                       #  A/B 1.0/20.0
model = model.to('cuda')
dev_texts = df_test_reddit['text'].tolist()                                                                                    # reddit/twitter
dev_dataset = TextDataset(tokenizer, dev_texts)
dev_loader = DataLoader(dev_dataset, batch_size=4, shuffle=False)
predictions = []
for batch in tqdm(dev_loader, desc="Evaluating"):
    inputs = {k: v.to(model.device) for k, v in batch.items() if k != 'labels'}
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    probabilities = torch.nn.functional.softmax(logits, dim=1)
    pred_labels = logits.argmax(1)
    predictions.extend(zip(pred_labels.cpu().numpy(), probabilities[:,0].cpu().numpy(), probabilities[:,1].cpu().numpy()))
df_predictions_15 = pd.DataFrame(predictions, columns=['predicted_label', 'prediction_0_weight', 'prediction_1_weight'])       #_1.0/_20.0
df_test_reddit.reset_index(drop=True, inplace=True)                                                                           # reddit/twitter
df_predictions_15.reset_index(drop=True, inplace=True)                                                                         #_1.0/_20.0
df_merged_15 = pd.concat([df_test_reddit, df_predictions_15], axis=1)                                                           # reddit/twitter #_1.0/_20.0 #_1.0/_20.0
df_merged_15.to_csv('BartLargeA_task6_predictions_069_epoch_15.csv', index=False)                                               # A/B            #_1.0/_20.0 #_1/_20

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'LABEL_0', '1': 'LABEL_1'}. The number of labels wil be overwritten to 2.


Evaluating:   0%|          | 0/1500 [00:00<?, ?it/s]

In [27]:
tokenizer = AutoTokenizer.from_pretrained('facebook/bart-large')
model = AutoModelForSequenceClassification.from_pretrained('/content/model_saves/epoch_bartlarge_A_16.0')                       #  A/B 1.0/20.0
model = model.to('cuda')
dev_texts = df_test_reddit['text'].tolist()                                                                                    # reddit/twitter
dev_dataset = TextDataset(tokenizer, dev_texts)
dev_loader = DataLoader(dev_dataset, batch_size=4, shuffle=False)
predictions = []
for batch in tqdm(dev_loader, desc="Evaluating"):
    inputs = {k: v.to(model.device) for k, v in batch.items() if k != 'labels'}
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    probabilities = torch.nn.functional.softmax(logits, dim=1)
    pred_labels = logits.argmax(1)
    predictions.extend(zip(pred_labels.cpu().numpy(), probabilities[:,0].cpu().numpy(), probabilities[:,1].cpu().numpy()))
df_predictions_16 = pd.DataFrame(predictions, columns=['predicted_label', 'prediction_0_weight', 'prediction_1_weight'])       #_1.0/_20.0
df_test_reddit.reset_index(drop=True, inplace=True)                                                                           # reddit/twitter
df_predictions_16.reset_index(drop=True, inplace=True)                                                                         #_1.0/_20.0
df_merged_16 = pd.concat([df_test_reddit, df_predictions_16], axis=1)                                                           # reddit/twitter #_1.0/_20.0 #_1.0/_20.0
df_merged_16.to_csv('BartLargeA_task6_predictions_069_epoch_16.csv', index=False)                                               # A/B            #_1.0/_20.0 #_1/_20

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'LABEL_0', '1': 'LABEL_1'}. The number of labels wil be overwritten to 2.


Evaluating:   0%|          | 0/1500 [00:00<?, ?it/s]

In [28]:
tokenizer = AutoTokenizer.from_pretrained('facebook/bart-large')
model = AutoModelForSequenceClassification.from_pretrained('/content/model_saves/epoch_bartlarge_A_17.0')                       #  A/B 1.0/20.0
model = model.to('cuda')
dev_texts = df_test_reddit['text'].tolist()                                                                                    # reddit/twitter
dev_dataset = TextDataset(tokenizer, dev_texts)
dev_loader = DataLoader(dev_dataset, batch_size=4, shuffle=False)
predictions = []
for batch in tqdm(dev_loader, desc="Evaluating"):
    inputs = {k: v.to(model.device) for k, v in batch.items() if k != 'labels'}
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    probabilities = torch.nn.functional.softmax(logits, dim=1)
    pred_labels = logits.argmax(1)
    predictions.extend(zip(pred_labels.cpu().numpy(), probabilities[:,0].cpu().numpy(), probabilities[:,1].cpu().numpy()))
df_predictions_17 = pd.DataFrame(predictions, columns=['predicted_label', 'prediction_0_weight', 'prediction_1_weight'])       #_1.0/_20.0
df_test_reddit.reset_index(drop=True, inplace=True)                                                                           # reddit/twitter
df_predictions_17.reset_index(drop=True, inplace=True)                                                                         #_1.0/_20.0
df_merged_17 = pd.concat([df_test_reddit, df_predictions_17], axis=1)                                                           # reddit/twitter #_1.0/_20.0 #_1.0/_20.0
df_merged_17.to_csv('BartLargeA_task6_predictions_069_epoch_17.csv', index=False)                                               # A/B            #_1.0/_20.0 #_1/_20

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'LABEL_0', '1': 'LABEL_1'}. The number of labels wil be overwritten to 2.


Evaluating:   0%|          | 0/1500 [00:00<?, ?it/s]

In [29]:
tokenizer = AutoTokenizer.from_pretrained('facebook/bart-large')
model = AutoModelForSequenceClassification.from_pretrained('/content/model_saves/epoch_bartlarge_A_18.0')                       #  A/B 1.0/20.0
model = model.to('cuda')
dev_texts = df_test_reddit['text'].tolist()                                                                                    # reddit/twitter
dev_dataset = TextDataset(tokenizer, dev_texts)
dev_loader = DataLoader(dev_dataset, batch_size=4, shuffle=False)
predictions = []
for batch in tqdm(dev_loader, desc="Evaluating"):
    inputs = {k: v.to(model.device) for k, v in batch.items() if k != 'labels'}
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    probabilities = torch.nn.functional.softmax(logits, dim=1)
    pred_labels = logits.argmax(1)
    predictions.extend(zip(pred_labels.cpu().numpy(), probabilities[:,0].cpu().numpy(), probabilities[:,1].cpu().numpy()))
df_predictions_18 = pd.DataFrame(predictions, columns=['predicted_label', 'prediction_0_weight', 'prediction_1_weight'])       #_1.0/_20.0
df_test_reddit.reset_index(drop=True, inplace=True)                                                                           # reddit/twitter
df_predictions_18.reset_index(drop=True, inplace=True)                                                                         #_1.0/_20.0
df_merged_18 = pd.concat([df_test_reddit, df_predictions_18], axis=1)                                                           # reddit/twitter #_1.0/_20.0 #_1.0/_20.0
df_merged_18.to_csv('BartLargeA_task6_predictions_069_epoch_18.csv', index=False)                                               # A/B            #_1.0/_20.0 #_1/_20

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'LABEL_0', '1': 'LABEL_1'}. The number of labels wil be overwritten to 2.


Evaluating:   0%|          | 0/1500 [00:00<?, ?it/s]

In [30]:
tokenizer = AutoTokenizer.from_pretrained('facebook/bart-large')
model = AutoModelForSequenceClassification.from_pretrained('/content/model_saves/epoch_bartlarge_A_19.0')                       #  A/B 1.0/20.0
model = model.to('cuda')
dev_texts = df_test_reddit['text'].tolist()                                                                                    # reddit/twitter
dev_dataset = TextDataset(tokenizer, dev_texts)
dev_loader = DataLoader(dev_dataset, batch_size=4, shuffle=False)
predictions = []
for batch in tqdm(dev_loader, desc="Evaluating"):
    inputs = {k: v.to(model.device) for k, v in batch.items() if k != 'labels'}
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    probabilities = torch.nn.functional.softmax(logits, dim=1)
    pred_labels = logits.argmax(1)
    predictions.extend(zip(pred_labels.cpu().numpy(), probabilities[:,0].cpu().numpy(), probabilities[:,1].cpu().numpy()))
df_predictions_19 = pd.DataFrame(predictions, columns=['predicted_label', 'prediction_0_weight', 'prediction_1_weight'])       #_1.0/_20.0
df_test_reddit.reset_index(drop=True, inplace=True)                                                                           # reddit/twitter
df_predictions_19.reset_index(drop=True, inplace=True)                                                                         #_1.0/_20.0
df_merged_19 = pd.concat([df_test_reddit, df_predictions_19], axis=1)                                                           # reddit/twitter #_1.0/_20.0 #_1.0/_20.0
df_merged_19.to_csv('BartLargeA_task6_predictions_069_epoch_19.csv', index=False)                                               # A/B            #_1.0/_20.0 #_1/_20

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'LABEL_0', '1': 'LABEL_1'}. The number of labels wil be overwritten to 2.


Evaluating:   0%|          | 0/1500 [00:00<?, ?it/s]

In [31]:
tokenizer = AutoTokenizer.from_pretrained('facebook/bart-large')
model = AutoModelForSequenceClassification.from_pretrained('/content/model_saves/epoch_bartlarge_A_20.0')                       #  A/B 1.0/20.0
model = model.to('cuda')
dev_texts = df_test_reddit['text'].tolist()                                                                                    # reddit/twitter
dev_dataset = TextDataset(tokenizer, dev_texts)
dev_loader = DataLoader(dev_dataset, batch_size=4, shuffle=False)
predictions = []
for batch in tqdm(dev_loader, desc="Evaluating"):
    inputs = {k: v.to(model.device) for k, v in batch.items() if k != 'labels'}
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    probabilities = torch.nn.functional.softmax(logits, dim=1)
    pred_labels = logits.argmax(1)
    predictions.extend(zip(pred_labels.cpu().numpy(), probabilities[:,0].cpu().numpy(), probabilities[:,1].cpu().numpy()))
df_predictions_20 = pd.DataFrame(predictions, columns=['predicted_label', 'prediction_0_weight', 'prediction_1_weight'])       #_1.0/_20.0
df_test_reddit.reset_index(drop=True, inplace=True)                                                                           # reddit/twitter
df_predictions_20.reset_index(drop=True, inplace=True)                                                                         #_1.0/_20.0
df_merged_20 = pd.concat([df_test_reddit, df_predictions_20], axis=1)                                                           # reddit/twitter #_1.0/_20.0 #_1.0/_20.0
df_merged_20.to_csv('BartLargeA_task6_predictions_069_epoch_20.csv', index=False)                                               # A/B            #_1.0/_20.0 #_1/_20

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'LABEL_0', '1': 'LABEL_1'}. The number of labels wil be overwritten to 2.


Evaluating:   0%|          | 0/1500 [00:00<?, ?it/s]

In [32]:
df_merged_20['predicted_label'].value_counts()

predicted_label
1    3694
0    2306
Name: count, dtype: int64

In [33]:
df_merged_20

,id,text,source,predicted_label,prediction_0_weight,prediction_1_weight
0,13001,"so first off, i *have* made an appointment wit...",reddit,1,0.000003,0.999997
1,13002,per the clinic ophthalmologist i have monocula...,reddit,1,0.000002,0.999998
2,13003,top 10 questions asked about cataract:,reddit,0,0.999991,0.000009
3,13004,"hi, everyone :-)i have had keratoconus in 15 y...",reddit,0,0.999996,0.000004
4,13005,ivmed-80. not a lot of literature on it and it...,reddit,0,0.999996,0.000004
...,...,...,...,...,...,...
5995,21196,therapist!! if you can. i know i had troubles ...,reddit,0,0.999993,0.000007
5996,21197,the low self-esteem is a major cause of your s...,reddit,0,0.999996,0.000004
5997,21198,"i am 21 and am good looking, smart but also in...",reddit,1,0.000002,0.999998
5998,21199,"youve really got to put in the work, ive been ...",reddit,0,0.999990,0.000010


In [37]:
submission_part_A = df_merged_20[['id','source','predicted_label']]
submission_part_A = submission_part_A.rename(columns={'predicted_label': 'label'})
submission_part_A.to_csv('submission_part_A.csv', index=False)

In [38]:
submission_part_A

,id,source,label
0,13001,reddit,1
1,13002,reddit,1
2,13003,reddit,0
3,13004,reddit,0
4,13005,reddit,0
...,...,...,...
5995,21196,reddit,0
5996,21197,reddit,0
5997,21198,reddit,1
5998,21199,reddit,0


In [39]:
!pip install transformers huggingface_hub

In [40]:
from huggingface_hub import notebook_login
notebook_login()

In [41]:
from huggingface_hub import HfApi, create_repo, upload_folder
hf_username = "1024m"  # Replace with your actual username
repo_name = "SMM4H-Task6-BartL-A20"  # Name of the new repository
api = HfApi()
if not api.repo_exists(f"{hf_username}/{repo_name}"):
    create_repo(repo_name,private=False)
model_directory = '/content/model_saves/epoch_bartlarge_A_20.0'
upload_folder(
    folder_path=model_directory,
    repo_id=f"{hf_username}/{repo_name}"
)

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/1024m/SMM4H-Task6-BartL-A20/commit/e740fc5adb351e68249ea140f7314acbd6763879', commit_message='Upload folder using huggingface_hub', commit_description='', oid='e740fc5adb351e68249ea140f7314acbd6763879', pr_url=None, pr_revision=None, pr_num=None)